<a href="https://colab.research.google.com/github/mark-6941/AAAAAAAA/blob/master/Quantam_llm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import paddle
try:
    import paddle_quantum
    print("Paddle Quantum 導入成功")
except ImportError:
    print("請先執行 !pip install paddle-quantum")

import numpy as np
from paddle.vision.datasets import MNIST
from paddle.vision.transforms import ToTensor

# 0. 設備設定
# 優先嘗試 NPU，若失敗則回退至 CPU
try:
    paddle.device.set_custom_device('custom_npu')
    print("目前使用設備: custom_npu")
except:
    paddle.set_device('cpu')
    print("環境不支援 custom_npu，已切換至 CPU")

# 1. 定義數據集與 DataLoader
transform = ToTensor()
train_dataset = MNIST(mode='train', transform=transform)

train_loader = paddle.io.DataLoader(
    train_dataset,
    batch_size=64,
    shuffle=True,
    drop_last=True
)

# 2. 定義模型 (經典部分範例)
model = paddle.nn.Sequential(
    paddle.nn.Flatten(),
    paddle.nn.Linear(784, 10)
)

# 3. 執行數據迭代測試
print("開始測試數據讀取...")
for batch_id, (data, label) in enumerate(train_loader):
    # data 自動會在 set_custom_device 指定的設備上
    x_batch = paddle.flatten(data, start_axis=1)

    # 模型預測
    y_pred = model(x_batch)

    if batch_id == 0:
        print(f"成功處理第一組 Batch!")
        print(f"數據形狀: {x_batch.shape}, 預測結果形狀: {y_pred.shape}")
        break

In [2]:
import paddle
import numpy as np

# --- 模擬設備設定 ---
try:
    paddle.device.set_custom_device('custom_npu')
    print("🔋 雲端儲能系統已連接至 NPU 加速器")
except:
    paddle.set_device('cpu')
    print("⚠️ 未偵測到 NPU，使用 CPU 運行儲能模擬")

# --- 1. 儲能數據產生器 (模擬真實感測器數據) ---
# 假設欄位：[電壓V, 電流I, 溫度T, 內部阻抗R]
def get_ess_sensor_data(batch_size=32):
    # 這裡模擬儲能櫃上傳的實時數據
    x = np.random.rand(batch_size, 4).astype('float32')
    # 標籤：SOH (State of Health)
    y = np.random.uniform(0.7, 1.0, (batch_size, 1)).astype('float32')
    return paddle.to_tensor(x), paddle.to_tensor(y)

# --- 2. 量子增強儲能預測模型 (解耦設計) ---
class ESSQuantumInference(paddle.nn.Layer):
    def __init__(self):
        super().__init__()
        # 經典編碼層：將感測器數據映射到高維空間 (類似量子編碼)
        self.encoder = paddle.nn.Linear(4, 16)

        # 模擬量子變分電路 (VQC) 的作用
        # 這裡利用多層非線性變換模擬量子糾纏的效果
        self.quantum_proxy = paddle.nn.Sequential(
            paddle.nn.Tanh(),
            paddle.nn.Linear(16, 16),
            paddle.nn.ReLU()
        )

        # 輸出層：預測雲端剩餘壽命或 SOC
        self.decoder = paddle.nn.Linear(16, 1)

    def forward(self, x):
        encoded = self.encoder(x)
        # 模擬量子干涉過程
        q_features = self.quantum_proxy(encoded)
        return self.decoder(q_features)
    def forward(self, F):
        # 前向传播逻辑
        x = F.relu(self.fc1(x))  # 第一层 + ReLU 激活
        x = self.fc2(x)          # 第二层
        return x

    def __init__(self,nn):
        super().__init__()
        self.fc1 = nn.Linear(10, 5)  # 第一层：输入特征10个，输出特征5个
        self.relu = nn.ReLU()        # 非线性激活函数ReLU
        self.fc2 = nn.Linear(5, 1)   # 第二层：输入特征5个，输出特征1个

    def forward(self, x):
        x = self.fc1(x)  # 数据通过第一层
        x = self.relu(x) # 应用ReLU激活函数
        x = self.fc2(x)  # 数据通过第二层
        return x



# --- 3. 測試執行 ---
model = ESSQuantumInference()
x, y = get_ess_sensor_data()
pred = model(x)

print(f"--- 儲能系統預測完成 ---")
print(f"輸入特徵 (第一筆數據): {x[0].numpy()}")
print(f"預測 SOH (電池健康度): {pred[0].item():.10f}")

⚠️ 未偵測到 NPU，使用 CPU 運行儲能模擬


TypeError: ESSQuantumInference.__init__() missing 1 required positional argument: 'nn'

In [4]:
import torch
import torch.nn as nn

class SimpleNet(nn.Module):
    def __init__(self):
        super(SimpleNet, self).__init__()
        # 1. 定義網路層
        # 假設輸入維度為 4 (ESS 感測器數據)，輸出維度為 1 (SOH 健康度)
        self.fc1 = nn.Linear(4, 16)   # 第一層全連接
        self.relu = nn.ReLU()         # 激活函數
        self.fc2 = nn.Linear(16, 1)   # 輸出層

    def forward(self, x):
        # 2. 定義前向傳播過程
        x = self.fc1(x)
        x = self.relu(x)
        output = self.fc2(x)          # 計算 output
        return output                 # 現在 output 已經定義，可以回傳了

# --- 測試模型 ---
model = SimpleNet()
# 模擬一組儲能數據 [Batch_size=1, Features=4]
mock_input = torch.randn(1, 4)
prediction = model(mock_input)

print(f"雲端儲能系統預測結果: {prediction.item():.4f}")

In [1]:

import snap7
from snap7.util import get_real  # 必須明確導入，否則會紅字報錯
import torch
import torch.nn as nn
import numpy


from pymodbus.client import ModbusTcpClient
from pymodbus.exceptions import ModbusIOException

# PLC 连接参数
PLC_IP = "192.168.0.10"  # PLC IP 地址
PLC_PORT = 502           # Modbus TCP 默认端口

# 连接 PLC
client = ModbusTcpClient(PLC_IP, port=PLC_PORT)
if not client.connect():
    print("❌ 无法连接到 PLC，请检查网络和 IP 设置")
    exit(1)

# 初始化 PLC 通訊
plc = snap7.client.Client()

try:
    # 這裡的 IP 需與您的 Siemens PLC (S7-1200/1500) 一致
    # Rack 0, Slot 1 是 S7-1200/1500 的標準設定
    plc.connect('192.168.0.1', 0, 1)
    print("✅ 已成功連接 Siemens PLC")
except Exception as e:
    print(f"❌ 連接失敗: {e}")

# 读取保持寄存器（Holding Registers），假设地址 0 开始，读取 10 个
    rr = client.read_holding_registers(address=0, count=10, unit=1)

    if isinstance(rr, ModbusIOException) or rr.isError():
        print("❌ 读取 PLC 数据失败")
    else:
        # 转换为 NumPy 数组
        data = np.array(rr.registers, dtype=np.int16)
        print("📥 原始数据:", data)

def run_ai_with_plc():
    if not plc.get_connected():
        print("PLC 未連接")
        return

    # A. 讀取 PLC 訊號 (DB1, 起始位址 0, 長度 16 bytes)
    # 這 16 bytes 將被解析為 4 個 32-bit Float
    try:
        data = plc.db_read(1, 0, 16)

        # 使用 snap7.util.get_real 正確解析
        v = get_real(data, 0)   # Offset 0: 電壓
        i = get_real(data, 4)   # Offset 4: 電流
        t = get_real(data, 8)   # Offset 8: 溫度
        r = get_real(data, 12)  # Offset 12: 內阻

        print(f"📡 讀取晶片訊號 -> V: {v:.2f}, I: {i:.2f}, T: {t:.2f}, R: {r:.2f}")

        # 這裡可以接上您之前的 AI 模型進行推理...
        # input_tensor = torch.tensor([[v, i, t, r]], dtype=torch.float32)

    except Exception as e:
        print(f"數據讀取失敗: {e}")

# 執行測試
if plc.get_connected():
    run_ai_with_plc()

ERROR:snap7.common:b' TCP : Unreachable peer'


❌ 連接失敗: b' TCP : Unreachable peer'


In [2]:
pip install python-snap7

In [7]:
pip install python-snap7

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 145.1/145.1 kB 5.6 MB/s eta 0:00:00


In [ ]:
import paddle
import numpy as np

# 模擬儲能數據獲取
def get_ess_vqc_data():
    # 假設從電池管理系統 (BMS) 獲取數據
    # [電壓, 電流, 溫度, 內阻] -> 標籤: 1 (需要均衡充電), 0 (不需處理)
    ess_raw = np.array([
        [3.6, 0.5, 25, 12, 0],
        [3.2, 0.8, 45, 15, 1],
        # ... 更多實時數據
    ])
    data = ess_raw[:, :4]  # 前 4 位是感測器特徵
    label = ess_raw[:, 4]  # 最後一位是控制決策
    return data, label

# 在 run2 裡面整合
def run_ess_optimization():
    # model = QModel(4, ...) # 4 個輸入對應 4 個感測器
    # ... 訓練過程 ...
    # 當 result > 0.5 時，觸發儲能櫃均衡電路開關
    if result.item() > 0.5:
        print("警告：偵測到電池不平衡，啟動儲能均衡補償")

In [ ]:
# 嘗試從 GitHub 源碼安裝，這有機會繞過 PyPI 的版本鎖定
!pip install git+https://github.com/PaddlePaddle/Quantum.git

In [ ]:
# 安裝 Paddle Quantum 套件
!pip install paddle-quantum -i https://mirror.baidu.com/pypi/simple

In [ ]:
import sys
print(sys.version)

In [ ]:
!pip install paddle-quantum -i https://mirror.baidu.com/pypi/simple

In [ ]:
pip install paddlepaddle -U

In [ ]:
pip install paddlepaddle -i https://mirror.baidu.com/pypi/simple


In [ ]:
!docker build -t paddle-quantum-npu .

In [ ]:
RUN pip install --no-cache-dir paddlepaddle -i https://pypi.tuna.tsinghua.edu.cn/simple && \
    pip install --no-cache-dir "protobuf<3.21" --force-reinstall && \
    pip install --no-cache-dir paddle-quantum

In [ ]:
# 使用 %%writefile 將內容寫入實體檔案，而非執行它
%%writefile Dockerfile

FROM python:3.12-slim

# 安裝基礎系統工具
RUN apt-get update && apt-get install -y \
    ccache \
    curl \
    && rm -rf /var/lib/apt/lists/*

WORKDIR /app

# 依照正確順序安裝套件，解決您之前的 Protobuf 衝突
RUN pip install --no-cache-dir paddlepaddle -i https://pypi.tuna.tsinghua.edu.cn/simple && \
    pip install --no-cache-dir "protobuf<3.21" --force-reinstall && \
    pip install --no-cache-dir paddle-quantum

# 解決 TypeError 的關鍵環境變數
ENV PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION=python

COPY . /app
CMD ["python", "train.py"]

In [ ]:
import paddle
import paddle_quantum

# 檢查是否有 GPU，否則用 CPU
device_str = 'gpu' if paddle.is_compiled_with_cuda() else 'cpu'
paddle.set_device(device_str)
print(f"✅ 已切換至設備: {device_str}")


In [ ]:
import paddle
# 執行這行看輸出結果
print("目前可用的自定義設備:", paddle.device.get_all_custom_device_type())

In [ ]:
import paddle
import os

# 1. 檢查硬體文件是否存在於環境中
if os.path.exists('/dev/galcore') or os.path.exists('/dev/davinci0'):
    print("✅ 系統層：硬體設備文件已掛載")
else:
    print("❌ 系統層：找不到設備文件，請檢查 Docker/K8s 掛載設定")

if os.
# 2. 檢查 Paddle 驅動支持
supported_devices = paddle.device.get_all_custom_device_type()
print(f"Paddle 偵測到的硬體類型: {supported_devices}")

# 3. 嘗試指派
try:
    if 'npu' in supported_devices:
        paddle.set_device('npu')
        print("🚀 成功掛載並切換至 NPU 運算模式")
    else:
        paddle.set_device('cpu')
        print("⚠️ 未偵測到 NPU 插件，目前使用 CPU 運行")
except Exception as e:
    print(f"🚨 掛載出錯: {e}")

In [ ]:
# 1. 解決 Protobuf 相容性問題
!pip install "protobuf<3.21" --force-reinstall

In [ ]:
import os
os.environ['PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION'] = 'python'
import paddle
import paddle_quantum
print("✅ 成功解決衝突並載入量槳")

In [ ]:
import os
os.environ['PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION'] = 'python'
import paddle
import paddle_quantum

In [ ]:
device = 'gpu' if paddle.is_compiled_with_cuda() else 'cpu'
paddle.set_device(device)
print(f"✅ 目前使用 {device} 進行模擬運算")

In [ ]:
# 1. 降級 Protobuf 到相容版本 (3.20.x)
!pip install "protobuf<3.21" --force-reinstall

In [ ]:
import sys
!{sys.executable} -m pip install paddlepaddle paddle-quantum -i https://pypi.tuna.tsinghua.edu.cn/simple

In [ ]:
!pip install paddle-quantum

In [ ]:
!pip install paddlepaddle -i https://pypi.tuna.tsinghua.edu.cn/simple